In [1]:
from pathlib import Path

import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import xgboost as xgb

# Loading Data and Training Models

1. Linear Regression
2. Random Forest Regression
3. XG Boost Regressor
4. Random Forest Classification
5. XG Boost Classifier

In [ ]:
data = pd.read_csv('/Users/mackdelany/Documents/DSR Notebooks/NLP-stock-price-prediction/final_data.csv')

In [ ]:
data = data.dropna() 
data.head()

In [ ]:
y = data['Label'].values
X = data.drop('Label',axis=1).to_numpy()
X = X[:,:20000] #uncomment to run with NLP data only
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state = 0, shuffle=False)

In [ ]:
## shift y from 'today' to 'tomorrow' 
""""y = y[1:]
X = X[:-1,:]"""

In [ ]:
lr = LogisticRegression(random_state=0)
"""rf_classifier = RandomForestClassifier(1000, verbose=1, n_jobs=-1, max_depth=100)
xg_classifier = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 2, alpha = 10, n_estimators = 100)"""

In [ ]:
lr.fit(X, y)
"""rf_classifier.fit(X_train, y_train)
xg_classifier.fit(X_train, y_train)"""

In [ ]:
X.shape

# 1. Logistic Regression

In [ ]:
y_lr_pred_train = lr.predict(X_train)
y_lr_pred_test = lr.predict(X_test)

In [ ]:
print("Training Accuracy:",metrics.accuracy_score(y_train, y_lr_pred_train))
print()
print("Test Accuracy:",metrics.accuracy_score(y_test, y_lr_pred_test))

# 2. Random Forest Classification

In [ ]:
y_rfc_pred_train = rf_classifier.predict(X_train)
y_rfc_pred_test = rf_classifier.predict(X_test)

In [ ]:
print("Training Accuracy:",metrics.accuracy_score(y_train, y_rfc_pred_train))
print()
print("Test Accuracy:",metrics.accuracy_score(y_test, y_rfc_pred_test))

# 3. XG Boost Classification

In [ ]:
y_xgc_pred_train = xg_classifier.predict(X_train)
y_xgc_pred_test = xg_classifier.predict(X_test)

In [ ]:
print("Training Accuracy:",metrics.accuracy_score(y_train, y_xgc_pred_train))
print()
print("Test Accuracy:",metrics.accuracy_score(y_test, y_xgc_pred_test))

# 4. Ensemble Classification (Mode)

In [ ]:
y_class_pred_ensemble_train = (y_lr_pred_train + y_rfc_pred_train + y_xgc_pred_train) // 3
y_class_pred_ensemble_test = (y_lr_pred_test + y_rfc_pred_test + y_xgc_pred_test) // 3

In [ ]:
print("Training Accuracy:",metrics.accuracy_score(y_train, y_class_pred_ensemble_train))
print()
print("Test Accuracy:",metrics.accuracy_score(y_test, y_class_pred_ensemble_test))

In [ ]:
def is_positive(row):
    if row > 0:
        return 1
    else :
        return 0


test['label_check'] = test.Close - test.Open
test['label_check'] = test['label_check'].apply(is_positive)

In [ ]:
(test.label_y - test.label_check).sum()

In [ ]:
X_lr = X[:,:20000]

In [ ]:
lr.fit(X_lr, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_lr, y, test_size = 0.10, random_state = 0, shuffle=False)

In [ ]:
y_lr_pred_train = lr.predict(X_train)
y_lr_pred_test = lr.predict(X_test)

In [ ]:
print("Training Accuracy:",metrics.accuracy_score(y_train, y_lr_pred_train))
print()
print("Test Accuracy:",metrics.accuracy_score(y_test, y_lr_pred_test))

# 5. Ensemble with different data sets

In [ ]:
y_class_pred_ensemble_test = (y_lr_pred_test + y_rfc_pred_test + y_xgc_pred_test) // 3

In [ ]:
print("Test Accuracy:",metrics.accuracy_score(y_test, y_class_pred_ensemble_test))